In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

from utils import *
from data import *
from models import *

# torch.autograd.set_detect_anomaly(True)
torch.set_num_threads(4)

unable to import 'smart_open.gcs', disabling that module


In [2]:
# ********* MODIFY HERE *********
args = {
    'model_read_ckpt': './ckpts/model_to_read',
    'lm_emb_path': 'albert-xxlarge-v1', # language model name
    'pretrained_wv': './wv/glove.6B.100d.txt', # original GloVe embeddings
    'vocab_size': 400100, # GloVe contains 400,000 words
    'device': 'cpu',
}

In [3]:
if args['device'] is not None and args['device'] != 'cpu':
    torch.cuda.set_device(args['device'])
elif args['device'] is None:
    if torch.cuda.is_available():
        gpu_idx, gpu_mem = set_max_available_gpu()
        args['device'] = f"cuda:{gpu_idx}"
    else:
        args['device'] = "cpu"

In [4]:
# Load the config file of trained ckpt
with open(args['model_read_ckpt']+'.json', 'r') as f:
    config = Config(**json.load(f))
    
    # load language model to dynamically calculate the contextualized word embeddings
    config.lm_emb_path = args['lm_emb_path']
    # assign device
    config.device = args['device']

In [5]:
model = JointModel(config)
model.load_ckpt(args['model_read_ckpt'])

/home/jue_wang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


albert-xxlarge-v1 is not file, try load as bert model.
albert-xxlarge-v1 loaded successfully.
Note it only supports default options now, i.e.: 
  layers='-1,-2,-3,-4', use_scalar_mix=True, pooling_operation="mean"


In [6]:
# Load full GloVe embeddings 
# *this is needed when training on the reduced version of GloVe word vectors 
# *you can comment this block if the OOV problem is not very serious.
_w = model.token_embedding.token_embedding.weight 
_w_data = _w.data
_w.data = torch.zeros([args['vocab_size'], config.token_emb_dim], dtype=_w.dtype, device=_w.device)
model.token_embedding.load_pretrained(args['pretrained_wv'], freeze=True)
_w.data[:len(_w_data)] = _w_data

400001it [00:16, 24582.56it/s]


In [7]:
# Now predict on custom text
rets = model.predict_step({
    'tokens': [['My', 'name', 'is', 'Jackson', ',', 'and', 'I', 'live', 'in', 'Berlin', '.'],]
})

In [8]:
rets['entity_preds']

[[(0, 1, 'PER'), (3, 4, 'PER'), (6, 7, 'PER'), (9, 10, 'GPE')]]

In [9]:
rets['relation_preds']

[{(6, 7, 9, 10, 'PHYS')}]